In [1]:
import matplotlib.pyplot as plt

In [3]:
conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch


Note: you may need to restart the kernel to use updated packages.Collecting package metadata (current_repodata.json): ...working... done

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\manju\anaconda3

  added / updated specs:
    - cudatoolkit=10.2
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cudatoolkit-10.2.89        |       h74a9793_1       317.2 MB
    pytorch-1.10.2             |py3.9_cuda10.2_cudnn7_0       950.4 MB  pytorch
    torchaudio-0.10.2          |       py39_cu102         2.1 MB  pytorch
    torchvision-0.11.3         |       py39_cu102         8.5 MB  pytorch
    ------------------------------------------------------------
                                           Total: 

In [1]:
import torch
import torch.nn as nn
import string
import random
import sys
import unidecode

In [2]:
#device Configuration
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
#Get characters from string.printable
all_characters = string.printable
n_characters = len(all_characters)
all_characters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [5]:
#Read large text file 
file = unidecode.unidecode(open('C:/Users/manju/OneDrive/Desktop/Ruparna/names_set.txt').read())

In [6]:
len(file)

7303750

In [12]:
class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers,output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first = True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden, cell):
        out=self.embed(x)
        out, (hidden, cell)=self.lstm(out.unsqueeze(1),(hidden,cell))
        out = self.fc(out.reshape(out.shape[0], -1))
        return out, (hidden, cell)
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden, cell

In [15]:
class Generator:
    def __init__(self):
        self.chunk_len = 250
        self.epochs = 5
        self.batch_size = 1
        self.print_every = 50
        self.hidden_size = 256
        self.num_layers = 2
        self.lr = 0.003
        
    def char_tensor(self, string):
        tensor = torch.zeros(len(string)).long()
        for c in range(len(string)):
            tensor[c] = all_characters.index(string[c])
            
        return tensor
    
    def get_random_batch(self):
        start_idx = random.randint(0, len(file) - self.chunk_len)
        end_idx = start_idx + self.chunk_len + 1
        print(start_idx, end_idx)
        text_str = file[start_idx:end_idx]
        text_input = torch.zeros(self.batch_size, self.chunk_len)
        text_target = torch.zeros(self.batch_size, self.chunk_len)
        
        for i in range(self.batch_size):
            text_input[i,:] = self.char_tensor(text_str[:-1])
            text_input[i,:] = self.char_tensor(text_str[1:])
            
        return text_input.long(), text_target.long()
    
    def generate(self,initial_str='A',predict_len=100,temperature=0.85):
        hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)
        initial_input = self.char_tensor(initial_str)
        predicted = initial_str
        
        for p in range(len(initial_str)-1):
            _, (hidden, cell) = self.rnn(initial_input[p].view(1).to(device), hidden, cell)
            
        last_char = initial_input[-1]
        
        for p in range(predict_len):
            output, (hidden, cell) = self.rnn(last_char[p].view(1).to(device), hidden, cell)
            output_dist = output.data.view(-1).div(temperature).exp()
            top_char = torch.multinomial(output_dist, 1)[0]
            
            predicted_char = all_characters[top_char]
            predicted +=predicted_char
            last_char = self.char_tensor(predicted_char)
        
        return predicted
        
    def train(self):
        self.rnn = RNN(n_characters, self.hidden_size, self.num_layers, n_characters).to(device)
        
        optimizer = torch.optim.Adam(self.rnn.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()
        
        print('=> Starting Training')
        
        for epochs in range(1, self.epochs + 1):
            inp, target = self.get_random_batch()
            hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)
            
            self.rnn.zero_grad()
            loss = 0
            inp = inp.to(device)
            target = target.to(device)
            
            for c in range(self.chunk_len):
                output, (hidden, cell) = self.rnn(inp[:, c], hidden, cell)
                loss += criterion(output, target[:,c])
                
            loss.backward()
            optimizer.step()
            loss = loss.item() / self.chunk_len
            
            if epochs % self.print_every ==  0:
                print(f'loss: {loss}')
                print(self.generate(initial_str='A',predict_len=100,temperature=0.85))

In [16]:
gennames = Generator()
gennames.train()

=> Starting Training
5457445 5457696
3569112 3569363
4507145 4507396
5331260 5331511
3883189 3883440
